# Prep

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [68]:
from bson.objectid import ObjectId

def show_doc(db, collection, id):
    '''
    Finds a document by 'id' and prints contents to the console
    
    Parameters:
    --------
    db         : database name
    collection : mongodb collection
    id         : mongodb document id
    
    Returns:
    --------
    Prints first 100 symbols of each document's key to console
    '''
    
    doc = db[collection].find_one({'_id':ObjectId(id)})
    for k in doc:
        print(f"{k} : {str(doc[k])}")

# Some comands to keep dbs clean

In [ ]:
# deletes all 'meta' fields from all docs
# htmlCol.update({}, {$unset: {meta:1}}, false, true); # mongo shell comand
htmlCol.update({}, {'$unset': {'meta':1}}, multi=True) # pymongo way

In [ ]:
# leaves only unique documents by 'url' field

htmlCol.create_index(
    "url",
    unique=True
)

In [ ]:
# pymongo 'find' returns cursor that allows iterating through results
# calling first object [0] allows accessing the dictionary with results
# the ['html'] is the key in the dictionary
html = htmlCol.find({'url':'http://www.msnbc.com/velshi-ruhle/watch/jeff-sessions-is-justifying-harsh-immigration-policy-with-the-bible-1256689731629'},\
            projection={'html':True, '_id':False})[0]['html']

In [ ]:
# find documents NOT containing a 'tag': regex expression
import re
tag = re.compile('dek___3AQpw.')
docs = htmlCol.find({"html" : {'$not': tag}})
for d in docs[:20]: print(d['url'])

# Multithreading

In [ ]:
from multiprocessing import Process

# use multiprocessing to extract features
def func():
    DB_NAME = 'scrape'
    db = pm.MongoClient(host='localhost', port=27017, maxPoolSize=500)[DB_NAME]

    for collection in ['left','right']: docs_parser(db[collection])

proc = Process(target=func)
proc.start()

# **Production code**

In [ ]:
# For debugging turn on logging to console
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

# dependencies
import pymongo as pm

from resources.newspaper import newspaper
from resources.newspaper.newspaper.source import Source
from resources.newspaper.newspaper.article import Article

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
# DB_NAME = 'scrape'
DB_NAME = 'FINALP'
db = client[DB_NAME]

In [ ]:
SOURCES = {
    'left'  : [
        'https://newrepublic.com',
        'https://www.motherjones.com'
# 3. Slate
# 4. The Intercept
# 5. Daily Beast
# 6. The Atlantic
# 7. Washington Post
# 8. Politico
# 9. The Guardian
# 10. BBC
    ],
    'right' : [
        'https://www.breitbart.com'
# 2. Fox News
# 3. New York Post
# 4. The American Conservative
# 5. Washington Times
# 6. Daily Wire
# 7. The Fiscal Times
# 8. The Hill
# 9. The Daily Caller
# 10. Reason
    ]
}

In [29]:
def saveToDB(db, collection, url, html, meta={}):
    """
    Saves a document to mongoDB, making sure there are no duplicates by 
    'url' value
    
    Parameters:
    --------
    db, collection  : mongo db connection
    url, html, meta : values to store
    
    Returns:
    --------
    Saved document
    """
    collection = db[collection]
    collection.update_one(
        {'url' : url},
        {
            '$set':
                {'url' : url,
                 'html' : html,
                 'meta' : meta
                }
        }
        ,
        upsert=True
    )

In [64]:
def scrape(url, earliest_date, db, collection):
    '''
    Scrapes an article from the 'url' up to the 'latest_date'
    
    Parameters:
    --------
    url         : main news website url
    date        : YYYY-MM-DD
    db          : database name
    collection  : mongodb collection
    
    Returns:
    --------
    Article's html and features stored to db, 
    Article's publish date
    
    '''
    article = Article(url)
    article.download()
    article.parse()
    
    saveToDB(db, collection, article.url, article.html, meta={
        'date'    :article.publish_date,
        'title'   :article.title,
        'text'    :article.text,
        'authors' :article.authors
    })
    
    return article.publish_date

# Newrepublic

In [62]:
import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser

# name of collection for this media
collection = 'newRep'
source = 'https://newrepublic.com/latest'
page   = 1

earliest_date = date_parser('2018-01-01')

while True:
    log.debug(f'PROCESSING PAGE: {page}')
    s = Source(source+'?page='+str(page))
    s.download()

    soup = bs.BeautifulSoup(s.html,'lxml')

    for section in soup.findAll('article'):
        url = urljoin(s.url, section.a['href'])
        log.debug(f'Processing url: {url}')
        
        article_date = scrape(url, earliest_date, db, collection)

    if article_date < earliest_date:
        log.debug(f'Reached earliest date requested: {article_date}')
        break
    page += 1

DEBUG:__main__:PROCESSING PAGE: 1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /latest?page=1 HTTP/1.1" 200 78494
DEBUG:__main__:Processing url: https://newrepublic.com/article/150202/trumps-farmer-bailout-americas-broken-food-system
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/150202/trumps-farmer-bailout-americas-broken-food-system HTTP/1.1" 200 20173
DEBUG:__main__:Processing url: https://newrepublic.com/article/150169/zadie-smiths-right-wrong
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/150169/zadie-smiths-right-wrong HTTP/1.1" 200 20603
DEBUG:__main__:Processing url: https://newrepublic.com/article/150159/doxx-racist
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1

DEBUG:__main__:Processing url: https://newrepublic.com/article/149947/white-house-pr-nightmare-never-came
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149947/white-house-pr-nightmare-never-came HTTP/1.1" 200 20967
DEBUG:__main__:Processing url: https://newrepublic.com/article/149955/trump-not-putins-puppet
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149955/trump-not-putins-puppet HTTP/1.1" 200 19697
DEBUG:__main__:Processing url: https://newrepublic.com/article/149950/russians-saw-helsinki-summit
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149950/russians-saw-helsinki-summit HTTP/1.1" 200 19777
DEBUG:__main__:Processing url: https://newrepublic.com/article/149939/american

DEBUG:__main__:PROCESSING PAGE: 6
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /latest?page=6 HTTP/1.1" 200 60122
DEBUG:__main__:Processing url: https://newrepublic.com/article/149717/watershed-moment-american-history-trump-nominates-brett-kavanaugh
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149717/watershed-moment-american-history-trump-nominates-brett-kavanaugh HTTP/1.1" 200 20686
DEBUG:__main__:Processing url: https://newrepublic.com/article/149683/trump-might-lose-trade-war-china
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149683/trump-might-lose-trade-war-china HTTP/1.1" 200 19429
DEBUG:__main__:Processing url: https://newrepublic.com/article/149628/americas-enduring-failure

DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149491/1990s-got-wrong HTTP/1.1" 200 22743
DEBUG:__main__:Processing url: https://newrepublic.com/article/149438/big-pharma-captured-one-percent
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149438/big-pharma-captured-one-percent HTTP/1.1" 200 42370
DEBUG:__main__:Processing url: https://newrepublic.com/article/149433/glow-gets-empowered
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149433/glow-gets-empowered HTTP/1.1" 200 20848
DEBUG:__main__:Processing url: https://newrepublic.com/article/149437/echoes-chinese-exclusion
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149437/echoes-chinese-exclusion HTTP/1.1

DEBUG:__main__:Processing url: https://newrepublic.com/article/149309/wilbur-rosss-financial-shenanigans-massive-scandal
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/149309/wilbur-rosss-financial-shenanigans-massive-scandal HTTP/1.1" 200 19593
DEBUG:__main__:Processing url: https://newrepublic.com/article/148864/democrats-mayors-presidential-candidates
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/148864/democrats-mayors-presidential-candidates HTTP/1.1" 200 23787
DEBUG:__main__:PROCESSING PAGE: 11
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /latest?page=11 HTTP/1.1" 200 98360
DEBUG:__main__:Processing url: https://newrepublic.com/article/149305/atttime-warner-merger-already-governm

...

DEBUG:__main__:Processing url: https://newrepublic.com/article/146959/truth-appalachia
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146959/truth-appalachia HTTP/1.1" 200 19875
DEBUG:__main__:PROCESSING PAGE: 70
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /latest?page=70 HTTP/1.1" 200 75269
DEBUG:__main__:Processing url: https://newrepublic.com/article/146952/elizabeth-warren-model-political-leadership
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146952/elizabeth-warren-model-political-leadership HTTP/1.1" 200 20527
DEBUG:__main__:Processing url: https://newrepublic.com/article/146945/waco-perfect-show-trump-erain-bad-way
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146821/lively-irrelevance-conservative-magazines HTTP/1.1" 200 21585
DEBUG:__main__:Processing url: https://newrepublic.com/article/146796/scientists-saved-bear-fish-skinand-stumbled-game-changing-idea
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146796/scientists-saved-bear-fish-skinand-stumbled-game-changing-idea HTTP/1.1" 200 22658
DEBUG:__main__:Processing url: https://newrepublic.com/article/146820/man-us-steel
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146820/man-us-steel HTTP/1.1" 200 20046
DEBUG:__main__:Processing url: https://newrepublic.com/article/146806/democrats-can-learn-cecile-richards
DEBUG:urllib3.connectionpo

DEBUG:__main__:Processing url: https://newrepublic.com/article/146691/trump-likely-finish-first-term
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146691/trump-likely-finish-first-term HTTP/1.1" 200 22523
DEBUG:__main__:Processing url: https://newrepublic.com/article/146683/trump-fox-news-mainstreaming-white-nationalism
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146683/trump-fox-news-mainstreaming-white-nationalism HTTP/1.1" 200 19335
DEBUG:__main__:PROCESSING PAGE: 75
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /latest?page=75 HTTP/1.1" 200 67098
DEBUG:__main__:Processing url: https://newrepublic.com/article/146677/trumps-middle-east-policy-goodish-bad-ugly
DEBUG:urllib3.connecti

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146589/mormonisms-crisis-faith HTTP/1.1" 200 22513
DEBUG:__main__:Processing url: https://newrepublic.com/article/146600/trumps-alarming-definition-treason
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146600/trumps-alarming-definition-treason HTTP/1.1" 200 19042
DEBUG:__main__:Processing url: https://newrepublic.com/article/146596/trumps-global-bigotry
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146596/trumps-global-bigotry HTTP/1.1" 200 20186
DEBUG:__main__:Processing url: https://newrepublic.com/article/146598/conservatives-trustbusters-now
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:u

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146501/afterlife-steve-bannon HTTP/1.1" 200 22387
DEBUG:__main__:Processing url: https://newrepublic.com/article/146507/trumps-voter-fraud-crusade-just-beginning
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146507/trumps-voter-fraud-crusade-just-beginning HTTP/1.1" 200 18994
DEBUG:__main__:Processing url: https://newrepublic.com/article/146505/michael-wolffs-revelations
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.com:443 "GET /article/146505/michael-wolffs-revelations HTTP/1.1" 200 19628
DEBUG:__main__:PROCESSING PAGE: 80
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): newrepublic.com
DEBUG:urllib3.connectionpool:https://newrepublic.c

In [96]:
# update text field to include more data
for doc in db[collection].find():
    soup = bs.BeautifulSoup(doc['html'],'lxml')
    text = ''
    for div in soup.findAll('div',{"class": "content-body"}):
        text += div.text
    if len(doc['meta']['text']) < len(text):
        db[collection].update_one(
            {'url' : doc['url']},
            {
                '$set':
                    {
                     'meta.text' : text
                    }
            }
            ,
            upsert=True
        )

In [97]:
url = 'https://newrepublic.com/article/146927/bad-dreams'
doc = db[collection].find_one({'url':url})

# The Atlantic

In [ ]:
import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser
from time import sleep
import random

# name of collection for this media
collection = 'theAtlantic'
source = 'https://www.theatlantic.com/latest/'
page   = 1

earliest_date = date_parser('2018-07-01')

while True:
    log.debug(f'PROCESSING PAGE: {page}')
    s = Source(source+'?page='+str(page))
    s.download()

    soup = bs.BeautifulSoup(s.html,'lxml')

    for section in soup.findAll('li', {"class":"article"}):
        url = urljoin(s.url, section.a['href'])
        log.debug(f'Processing url: {url}')
        
        article_date = scrape(url, earliest_date, db, collection)
        
        sleep(random.uniform(1,5))

    if article_date < earliest_date:
        log.debug(f'Reached earliest date requested: {article_date}')
        break
    page += 1

DEBUG:__main__:PROCESSING PAGE: 1
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /latest/?page=1 HTTP/1.1" 200 24879
DEBUG:__main__:Processing url: https://www.theatlantic.com/photo/2018/07/photos-made-in-china/566150/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /photo/2018/07/photos-made-in-china/566150/ HTTP/1.1" 200 32347
DEBUG:__main__:Processing url: https://www.theatlantic.com/education/archive/2018/07/conservative-high-schoolers-are-ready-to-own-the-libs/566177/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /education/archive/2018/07/conservative-high-schoolers-are-ready-to-own-the-libs/566177/ HTTP/1.1" 200 33645
DEBUG:__main__:Processing url: https://www.theatlantic.com

DEBUG:__main__:Processing url: https://www.theatlantic.com/politics/archive/2018/07/the-24-candidates-for-2018-sunk-by-metoo-allegations/565457/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /politics/archive/2018/07/the-24-candidates-for-2018-sunk-by-metoo-allegations/565457/ HTTP/1.1" 200 60672
DEBUG:__main__:Processing url: https://www.theatlantic.com/entertainment/archive/2018/07/batman-catwoman-and-the-marriage-plot-in-comics/565295/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /entertainment/archive/2018/07/batman-catwoman-and-the-marriage-plot-in-comics/565295/ HTTP/1.1" 200 35860
DEBUG:__main__:Processing url: https://www.theatlantic.com/international/archive/2018/07/afghanistan-us-taliban/565799/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlan

DEBUG:__main__:Processing url: https://www.theatlantic.com/international/archive/2018/07/trump-pressure-iran-rouhani/566000/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /international/archive/2018/07/trump-pressure-iran-rouhani/566000/ HTTP/1.1" 200 33028
DEBUG:__main__:Processing url: https://www.theatlantic.com/family/archive/2018/07/why-dont-more-men-take-their-wives-last-names/565898/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GET /family/archive/2018/07/why-dont-more-men-take-their-wives-last-names/565898/ HTTP/1.1" 200 32914
DEBUG:__main__:Processing url: https://www.theatlantic.com/business/archive/2018/07/women-umpires/564641/
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.theatlantic.com
DEBUG:urllib3.connectionpool:https://www.theatlantic.com:443 "GE